# The Spread of an Epidemic

### Introduction

Previously to the actual Corona virus SARS Cov-2 crisis, several epidemics occurred in history. Already in Antiquity, due to the lack of treatment and knowledge, diseases have decimated entire populations in few months or even few weeks. 
Nowadays, a scientific discipline named epidemiology study method to find the causes of health outcomes and diseases in populations. It specifically studies the distribution (frequency, pattern) and determinants (causes, risk factors) of health-related states and events in specified populations and bringing on solutions against epidemics and in particular for pandemic situations such as the Corona virus one we are experiencing lately. 

According to the Center for Disease Control and Prevention (CDC), the difference between an epidemic and a pandemic is that “epidemic is a sudden outbreak of a disease in a certain geographical area. Pandemic is an outbreak of a disease that has spread across several countries or continents.” One of the major concerns of this crisis is not only the lack of treatment but also the speed of spread.

Computer simulation models are well-established tools within epidemiology and related disciplines. They have proven useful, powerful, and accessible for evaluating questions about the spread of infectious disease within and between many populations and over physical and social spaces.

A useful measure for the spread of an epidemic is the  basic reproductive number a.k.a R0. It is the expected number of cases directly generated by one case, in a population where all individuals are susceptible to get infected. This measure depends on the contact rate and the recovery time. Computing R0 allows us  to predict the way an epidemic will evolve and thus to find solutions to slow down the contamination. In particular, if R0<1 then an infected individual infects less than one person on average, which means that the disease disappears from the population. Otherwise, if R0>1 then the disease spreads in the population and becomes epidemic.


### The simulation :

To model the spread of an epidemic, I chose to take a sample of 100 individuals where one is infected amongst them at T0. The subjects move like atoms i.e. they follow attractive/repulsive force. As we used the Lennard-Jones Potential in the previous exercise, it seems that it was too strong. Thus, i decided to modify it for a weaker one. 
In this model, there is 3 categories of people:
- Infected people in red
- Recovered people in green
- Susceptible of being contaminated in yellow (they haven't been in contact with the disease)

To undergo from the susceptible category to the infected category, each individual needs to be in a radius of less than a meter from an infected subject. Since there is still a probability to get or not infected, I left this decision to the appreciation of the reader.


In [74]:
from vpython import *
import numpy as np
import random

###Set the scene - a room with people###

N = 100 #number of people in the simulation
people = [0]*N
N_infected = 1 #number of people infected at the beginning of the simulation
N_recovery = 0 #number of people that recovered at the beginning of the simulation
R0_list = [0]*N
time_infected = [0]*N #check how long a person is sick
time_recovery = 7 #time it took to recover from he virus
prob_infected = 0.5 #the chance of getting the virus by an infected person
radius_infected = 1 #the maximum distance to has a probabilty to get infected
room_size  = 20 #Size of the room
room = canvas(background = color.white, fov = 0.01, width = 400, height = 400, center = vector(room_size/2,room_size/2,0), range = room_size/2, userspin = False, userzoom = False, autoscale = False)
x = [0]*N #coordonates of the people in the room
y = [0]*N
vx = [0]*N
vy = [0]*N
ax = [0]*N
ay = [0]*N
for i in range(len(people)): #Set randomly the people in the room
    x[i] = random.uniform(1,room_size-1)
    y[i] = random.uniform(1,room_size-1)
    people[i] = sphere( radius = 0.5, color = color.yellow, pos = vector(x[i] ,y[i] ,0) ) #a yellow ball mean a person that never add the virus
for j in range(N_infected) :
            people[j].color = color.red #a red ball meaning an infected person
        
    
    
###Set the movement of the people in the box###

def get_infected(people1,people2,infection_prob) :
    if (people1.color == color.red) and (people2.color == color.yellow) : #one is infected and the second is susceptible
        prob = random.randint(1,100)
        if (prob <= (infection_prob*100)) : #in the distance of a meter, a person has still a chance of having the virus
            people2.color = color.red
            return True
        else :
            return False
    else :
        return False

def compute_R0(people,R0_list):
    R = 0
    N_infectious = 0
    for i in range(len(people)) :
        if (people[i].color == color.red) :
            R += R0_list[i]
            N_infectious += 1
    if N_infectious > 0 :
        return R/N_infectious
    else :
        return 0
    
def computeAccelerations(x,y,vx,vy,people,infection_prob,radius_infected, room_size, R0_list) :
    ax = [0]*len(x)
    ay = [0]*len(y)
    for i in range(len(x)) :
        if (x[i] < 0.5 and vx[i]<0) or (x[i] > room_size-0.5 and vx[i]>0):
            vx[i]=-vx[i]
        if (y[i] < 0.5 and vy[i]<0) or (y[i] > room_size-0.5 and vy[i]>0):
            vy[i]=-vy[i]
        for j in range(i) :
            r = (x[i]-x[j])**2 + (y[i]-y[j])**2 
            if (r < 9) :
                fr = 1/(r**3 + 0.1) #MODFLO un peu plus gentil :) 4*( (12/(r*13)) - (6/(r*7)) )
                ax[i] += fr * (x[i]-x[j]) / r
                ay[i] += fr * (y[i]-y[j]) / r
                ax[j] += fr * (x[j]-x[i]) / r
                ay[j] += fr * (y[j]-y[i]) / r
                if (r <= radius_infected) :
                    if (get_infected(people[i],people[j],infection_prob)) :
                        R0_list[i] += 1
                    elif (get_infected(people[j],people[i],infection_prob)) :
                        R0_list[j] += 1
    return ax , ay

def singleStep(x,y,vx,vy,ax,ay,dt,people,infection_prob,radius_infected,room_size,R0_list) :
    ax,ay = computeAccelerations(x,y,vx,vy,people,infection_prob,radius_infected,room_size,R0_list) #MODFLO j'ai mis ça en premier, avant les mouvements
    for i in range (len(x)) :
        vy[i] += ay[i]*dt #MODFLO t'avais utilisé 0.5 ici, c'est faux, j'ai aussi mis ça avant le x, y pour que c'est déjà pris en compte dans la calculation suivante
        vx[i] += ax[i]*dt #MODFLO t'avais utilisé 0.5 ici, c'est faux
        y[i] += vy[i]*dt #MODFLO t'avais rajouté l'acceleration une deuxième fois ici, c'est pas necessaire à mon avis.
        x[i] += vx[i]*dt #MODFLO t'avais rajouté l'acceleration une deuxième fois ici, c'est pas necessaire à mon avis.
        
        
###Set the buttons and options###
def run():
    global running, start, time_recovery, prob_infected
    running = 1 - running
    if (start == 0) :
        start = 1
        time_recovery = time_recoverySlider.value
        prob_infected = prob_infectedSlider.value

def adjustTimeRecovery():
    time_recoverySliderReadout.text = str(time_recoverySlider.value) + " days"

def adjustProb():
    prob_infectedSliderReadout.text = str(prob_infectedSlider.value*100) + " %"

R0 = wtext( text="    Basic reproductive rate = " + str(compute_R0(people,R0_list)) )
room.append_to_caption("\n\n")
   
time_recoverySlider = slider(left=10, min=1, max=15, step=1, value=7, bind=adjustTimeRecovery)
room.append_to_caption("    Times to recover from the virus = ")
time_recoverySliderReadout = wtext(text="7 day")
room.append_to_caption("\n\n")

prob_infectedSlider = slider(left=10, min=0, max=1, step=0.01, value=1, bind=adjustProb)
room.append_to_caption("    Probability to get infected within a meter = ")
prob_infectedSliderReadout = wtext(text="100 %")
room.append_to_caption("\n\n")

graph(width=400, height=250, background = color.white, xtitle = "Time", ytitle = "Number of people", title = "Status of people - Pendemic Simulation")
gcurve()
infectedDots = gdots(interval = 10, color=color.red)
recoveredDots = gdots(interval = 10, color=color.green)
susceptibleDots = gdots(interval = 10, color = color.yellow)

###Set the simulation ###

dt = 0.02
t = 0
running = 0
start = 0
stepPerFrame = 1
button(text = "Play/Stop", bind = run)
ax,ay = computeAccelerations(x,y,vx,vy,people,prob_infected,radius_infected,room_size,R0_list)
while (N_infected > 0) :
    rate(1000)
    if (running) :
        N_infected = 0
        for step in range(stepPerFrame) : #call singleStep several times
            singleStep(x,y,vx,vy,ax,ay,dt,people,prob_infected,radius_infected,room_size,R0_list)
            t += dt*stepPerFrame
        for i in range(N) :
            people[i].pos = vector(x[i],y[i],0) #update the positions of all the person
            if (people[i].color == color.red) :
                time_infected[i] += dt*stepPerFrame
                if (time_infected[i] > time_recovery) :
                    people[i].color = color.green #If a person recovered from the virus the ball is green and can't infect people anymore
                    N_recovery += 1
                else :
                    N_infected += 1
        
        R0.text ="    Basic reproductive rate = " + str(compute_R0(people,R0_list))
        infectedDots.plot(t,N_infected, min = 0,max = N)
        recoveredDots.plot(t,N_recovery, min = 0,max = N)
        susceptibleDots.plot(t,N - N_infected - N_recovery, min = 0,max = N)
print("The virus can't infect people anymore.\nEnd of simulation.")

<IPython.core.display.Javascript object>

The virus can't infect people anymore.
End of simulation.


### The SIR model :

A simple mathematical description of the spread of a disease in a population is the so-called SIR model, which divides the (fixed) population of N individuals into three "compartments" which may vary as a function of time, t:

- S(t) are those susceptible but not yet infected with the disease;
- I(t) is the number of infectious individuals;
- R(t) are those individuals who have recovered from the disease and now have immunity to it.


The SIR model describes the change in the population of each of these compartments in terms of two parameters, β and γ. β describes the effective contact rate of the disease: an infected individual comes into contact with βN other individuals per unit time (of which the fraction that are susceptible to contracting the disease is S/N). γ is the mean recovery rate: that is, 1/γ is the mean period of time during which an infected individual can pass it on.

The differential equations describing this model were first derived by Kermack and McKendrick :

- dS/dt = −βSI/N 
- dI/dt = (βSI/N) − γI
- dR/dt = γI.

There are many modifications of the SIR model, including those that include births and deaths, where upon recovery there is no immunity (SIS model), where immunity lasts only for a short period of time (SIRS), where there is a latent period of the disease where the person is not infectious (SEIS and SEIR), and where infants can be born with immunity (MSIR).

In [11]:
from vpython import *
scene = canvas(background = color.white, width = 0, height = 0)

#Set the constant :
N = 100 #Fixed population
t = 0
dt = 0.01
tol = 10**(-4)
running = False

beta = 1 #contact rate 
gamma = 1/7 # 1/gamma is the period of recovery then gamma is the rate

I = 1 #Infectious people
dI = 0 #derivative of I

R = 0 #Recovered people
dR = 0 #derivative of R

S = N - I - R #Susceptible people
dS = 0 #derivative of S

#Set the widget and graph:
def run():
    global running, beta, gamma
    running = 1 - running
    gamma = 1/periodSlider.value
    beta = betaSlider.value
    R0.text = "    Basic reproductive rate = " + str(beta/gamma)
    scene.append_to_caption("\n\n")

def adjustPeriod():
    periodSliderReadout.text = str( periodSlider.value) + " days"

def adjustBeta():
    betaSliderReadout.text = str(betaSlider.value*100) + " %"

betaSlider = slider(left=10, min=0, max=1, step=0.01, value=1, bind=adjustBeta)
scene.append_to_caption("    Beta aka contact rate = ")
betaSliderReadout = wtext(text="100 %")
scene.append_to_caption("\n\n")


periodSlider = slider(left=10, min=1, max=15, step=1, value=7, bind=adjustPeriod)
scene.append_to_caption("    Period to recover aka 1/gamma = ")
periodSliderReadout = wtext(text="7 day")
scene.append_to_caption("\n\n")

R0 = wtext( text = "    Basic reproductive rate = " + str(beta / gamma))
scene.append_to_caption("\n\n")

graph(width=400, height=250, background = color.white, xtitle = "Time", ytitle = "Number of people", title = "SIR model")
gcurve()
infectedDots = gdots(interval = 10, color=color.red, label = "Infected", legend = True, radius = 0.5)
recoveredDots = gdots(interval = 10, color=color.green, label = "Recovered", legend = True, radius = 0.5)
susceptibleDots = gdots(interval = 10, color = color.yellow, label = "Susceptible", legend = True, radius = 0.5)

# Start the simulation :
button(text = "Play/Stop", bind = run)
while ( abs(- beta * S * I / N) > tol ) :
    rate(1000)
    
    if running :
        dS = - beta * S * I / N
        dI = - dS - gamma * I
        dR = gamma * I
    
        S += dS * dt
        I += dI * dt
        R += dR * dt 
    
        infectedDots.plot(t, I, min = 0,max = N)
        recoveredDots.plot(t, R, min = 0,max = N)
        susceptibleDots.plot(t, S, min = 0,max = N)
    
        t += dt

print("End of SIR Model.\n")

<IPython.core.display.Javascript object>

End of SIR Model.



### The SEIRS Model :

The SIR model assumes people carry lifelong immunity to a disease upon recovery, but for many diseases the immunity after infection wanes over time. In this case, the SEIRS model is used to allow recovered individuals (group R) to return to a susceptible state (group S). Specifically, ξ is the rate which recovered individuals return to the susceptible statue due to loss of immunity. Another group has been added in this model, the exposed state(group E). For most infectious diseases, there is a latent period between being infected and becoming infectious: the exposed group (E). Upon being infected (group I), individuals will move to this group at a rate βSI/N and remain there for an average period of 1/σ before moving into the I group.
If there is sufficient influx to the susceptible population, at equilibrium the dynamics will be in an endemic state with damped oscillation. A simple loop will helps us understand the different states that contain the SEIRS model :
Suscpetible → Exposed → Infectious → Recovered → Susceptible again

To make the model more realistic I decided to add vital dynamics, meaning that the population can fluctuate by adding a birth rate and a death rate. Where μ and ν represent the birth and death rates, respectively, and are assumed to be equal to maintain a constant population, the ODE then becomes:

- dS(t) / dt = μ N − ν S(t) − β S(t) I(t) / N
- dE(t) / dt = β S(t) I(t) / N − ν E(t) − σ E(t)
- dI(t) / dt = σ E(t) − γ I(t) − ν I(t)
- dR(t) / dt = γ I(t) − ν R(t)

where N=S+E+I+R is the total population.

In [9]:
from vpython import *
scene = canvas(background = color.white, width = 0, height = 0)

#Set the constant :
N = 100 #Fixed population
t = 0
dt = 0.01
tol = 10**(-4)
running = False

beta = 1 #contact rate 
gamma = 1/7 #1/gamma is the period of to recover then gamma is the rate
sigma = 1 #1/sigma is the incubation period, at the end of it the person become infectious
eta = 0 #1/eta is the period of time it takes to loose immunity
mu = 1/10
nu = 1/10

I = 1 #Infectious people
dI = 0 #derivative of I

E = 0 #Exposed people
dE = 0 #derivative of E

R = 0 #Recovered people
dR = 0 #derivative of R

S = N - I - R #Susceptible people
dS = 0 #derivative of S

#Set the widget and graph:
def run():
    global running, beta, gamma, eta, sigma, nu, mu
    running = 1 - running
    gamma = 1/gammaSlider.value
    beta = betaSlider.value
    eta = 1/etaSlider.value
    sigma = 1/sigmaSlider.value
    nu = vitalSlider.value
    mu = nu 
    R0.text = "    Basic reproductive rate = " + str(beta * sigma / ((nu + sigma) * (nu + gamma)))
    scene.append_to_caption("\n\n")

def adjustGamma():
    gammaSliderReadout.text = str( gammaSlider.value) + " days"

def adjustSigma():
    sigmaSliderReadout.text = str( sigmaSlider.value) + " days"

def adjustBeta():
    betaSliderReadout.text = str(betaSlider.value*100) + " %"
    
def adjustEta():
    etaSliderReadout.text = str(etaSlider.value) + " days"
    
def adjustVital():
    vitalSliderReadout.text = str(vitalSlider.value*100) + " % of the pop."

betaSlider = slider(left=10, min=0, max=1, step=0.01, value=1, bind=adjustBeta)
scene.append_to_caption("    Contact rate a.k.a β = ")
betaSliderReadout = wtext(text="100 %")
scene.append_to_caption("\n\n")

gammaSlider = slider(left=10, min=1, max=30, step=1, value=7, bind=adjustGamma)
scene.append_to_caption("    Period to recover a.k.a 1/γ = ")
gammaSliderReadout = wtext(text="7 day")
scene.append_to_caption("\n\n")

sigmaSlider = slider(left=10, min=1, max=15, step=1, value=3, bind=adjustSigma)
scene.append_to_caption("    Period of incubation a.k.a 1/σ = ")
sigmaSliderReadout = wtext(text="3 day")
scene.append_to_caption("\n\n")

etaSlider = slider(left=10, min=1, max=100, step=1, value=30, bind=adjustEta)
scene.append_to_caption("    Period of loss of immunity a.k.a 1/ξ = ")
etaSliderReadout = wtext(text="30 day")
scene.append_to_caption("\n\n")

vitalSlider = slider(left=10, min=0, max=1, step = 0.01, value = 0.1, bind=adjustVital)
scene.append_to_caption("    Rate of birth and death, respectively μ and ν = ")
vitalSliderReadout = wtext(text="10 %")
scene.append_to_caption("\n\n")

R0 = wtext( text = "    Basic reproductive rate = " + str(beta * sigma / ((nu + sigma) * (nu + gamma))) )
scene.append_to_caption("\n\n")

graph(width=400, height=250, background = color.white, xtitle = "Time", ytitle = "Number of people", title = "SEIRS model with vital dynamics")
gcurve()
infectedDots = gdots(interval = 10, color=color.red, label = "Infected", legend = False, radius = 0.5)
exposedDots = gdots(interval = 10, color = color.blue, label = "Exposed", legend = True, radius = 0.5)
recoveredDots = gdots(interval = 10, color=color.green, label = "Recovered", legend = False, radius = 0.5)
susceptibleDots = gdots(interval = 10, color = color.yellow, label = "Susceptible", legend = False, radius = 0.5)

# Start the simulation :
button(text = "Play/Stop", bind = run)
while ( abs(- beta * S * I / N + eta * R + mu * N - nu * S)>tol ) :
    rate(1000)
    
    if running :
        
        dS = - beta * S * I / N + eta * R + mu * N - nu * S
        dE = beta * S * I / N - sigma * E - nu * E
        dI = sigma * E - gamma * I - nu * I
        dR = gamma * I - eta * R - nu * R
        
        S += dS * dt
        E += dE * dt
        I += dI * dt
        R += dR * dt 
        
        infectedDots.plot(t, I, min = 0,max = N)
        exposedDots.plot(t,E, min = 0, max = N)
        recoveredDots.plot(t, R, min = 0,max = N)
        susceptibleDots.plot(t, S, min = 0,max = N)
    
        t += dt

print("End of SEIRS Model with vital dynamics.\n")

<IPython.core.display.Javascript object>

End of SEIRS Model with vital dynamics.



### An example : University 

### A model with solutions

The previous exposed models (SIR and SEIRS models) only allows us to stare at the epidemic. In these models, the population didn't take any preventive measures to slow down the spread of the epidemic or equivalently to minimize R0. Thus, the epidemic spread freely in the space. Let introduce a model in which the population try to find solution against the disease.

As mentionned before R0 depends on the contact rate and the time to recovery. Thus, two evident solutions to decelerate the spread of the disease will be, first of all, to quarantine people so the contact rate will decrease and also to vaccinate people so they will be immunize even before they get exposed to it.

Let extend our previous model so it will contain this two new states. To resume, the model considers seven stages of infection: susceptible (S), exposed (E), infectious (I), quarantined (Q), recovered (R), deaths (D), and vaccinated (V). The ODE of the SEIQRDV model are :

- dS/dt = μN − νS − βSI / N − αS
- dE(t)/ dt = βS(t)I(t) / N − νE(t) − σE(t) + λβV(t)I(t)/N
- dI(t)/ dt = σE(t) − δI(t) − νI(t)
- dQ(t)/dt = δI(t) − (1 − κ)γQ(t) − κρQ(t) − νQ(t)
- dR(t)/dt = (1 − κ)γQ(t) − νR(t)
- dD(t)/dt = κρQ(t)
- dV(t)/dt = αS(t) − λβV(t)I(t)/N − νV(t),


The coefficients are 
μ - birth rate ;
ν - natural death rate ;
α - vaccination rate ;
β - contact rate ;
1/σ - incubation period ;
1/δ - quarantine period ;
κ - mortality rate ;
1/γ - recovery period ;
ρ - average days until death ;
λ - the vaccine inefficacy (0 ≤ λ ≤ 1). So (1 − λ) represents the vaccine efficacy. If σ = 0, the vaccine offers 100% protection against the disease.

N = S + E + I + Q + R + D + V is the total population size.

In [16]:
from vpython import *
scene = canvas(background = color.white, width = 0, height = 0)

#Set the constant :
N = 100 #Fixed population
t = 0
dt = 0.01
tol = 10**(-3)
running = False

beta = 1 #contact rate 
gamma = 1/7 #1/gamma is the period of recovery then gamma is the rate
sigma = 1 #1/sigma is the incubation period, at the end of it the person become infectious
mu = 0.1 #birth rate
nu = mu #death rate (natural)
alpha = 0.6 #vaccination rate
delta = 0.6 # delta is the quarantine rate
k = 0.02 #mortality rate 
rho = 1/gamma #average days until death
l = 0.1 #vaccine inefficacy

eta = 1/2 #1/eta is the time it take to lost immunity

I = 1 #Infectious people
dI = 0 #derivative of I

E = 0 #Exposed people
dE = 0 #derivative of E

R = 0 #Recovered people
dR = 0 #derivative of R

S = N - I #Susceptible people
dS = 0 #derivative of S

V = 0 #Vaccinated people
dV = 0

Q = 0 #Quarantined people
dQ = 0

D = 0 #Dead people due to the disease
dD = 0

#Set the widget and graph:
def run():
    global running, beta, gamma, eta, sigma
    running = 1 - running
    gamma = 1/gammaSlider.value
    beta = betaSlider.value
    eta = 1/etaSlider.value
    sigma = 1/sigmaSlider.value
    nu = vitalSlider.value
    delta = deltaSlider.value
    mu = nu 
    R0.text = "    Basic reproductive rate = " + str(beta*sigma*mu*(nu+alpha*l)/(nu*(nu+sigma)*(nu+delta)*(nu+alpha)))
    scene.append_to_caption("\n\n")

def adjustGamma():
    gammaSliderReadout.text = str( gammaSlider.value) + " days"

def adjustSigma():
    sigmaSliderReadout.text = str( sigmaSlider.value) + " days"

def adjustBeta():
    betaSliderReadout.text = str(betaSlider.value*100) + " %"
    
def adjustEta():
    etaSliderReadout.text = str(etaSlider.value) + " days"
    
def adjustVital():
    vitalSliderReadout.text = str(vitalSlider.value*100) + " % of the pop."
    
def adjustAlpha():
    alphaSliderReadout.text = str(alphaSlider.value*100) + " % of the pop."

def adjustDelta():
    deltaSliderReadout.text = str(deltaSlider.value*100) + " % of the pop."
    
betaSlider = slider(left=10, min=0, max=1, step=0.01, value=1, bind=adjustBeta)
scene.append_to_caption("    Beta aka contact rate = ")
betaSliderReadout = wtext(text="100 %")
scene.append_to_caption("\n\n")


gammaSlider = slider(left=10, min=1, max=15, step=1, value=7, bind=adjustGamma)
scene.append_to_caption("    Period to recover aka 1/gamma = ")
gammaSliderReadout = wtext(text="7 day")
scene.append_to_caption("\n\n")

sigmaSlider = slider(left=10, min=1, max=15, step=1, value=3, bind=adjustSigma)
scene.append_to_caption("    Period of incubation aka 1/sigma = ")
sigmaSliderReadout = wtext(text="3 day")
scene.append_to_caption("\n\n")

etaSlider = slider(left=10, min=1, max=100, step=1, value=30, bind=adjustEta)
scene.append_to_caption("    Period of loss of immunity aka 1/eta = ")
etaSliderReadout = wtext(text="30 day")
scene.append_to_caption("\n\n")

vitalSlider = slider(left=10, min=0, max=1, step = 0.01, value = 0.1, bind=adjustVital)
scene.append_to_caption("    Rate of birth and death, respectively μ and ν = ")
vitalSliderReadout = wtext(text="10 %")
scene.append_to_caption("\n\n")

alphaSlider = slider(left=10, min=0, max=1, step = 0.01, value = 0.6, bind=adjustAlpha)
scene.append_to_caption("    Rate of vaccination aka α = ")
alphaSliderReadout = wtext(text="60 %")
scene.append_to_caption("\n\n")

deltaSlider = slider(left=10, min=0, max=1, step = 0.01, value = 0.6, bind=adjustDelta)
scene.append_to_caption("    Rate of quarantined aka δ = ")
deltaSliderReadout = wtext(text="60 %")
scene.append_to_caption("\n\n")

R0 = wtext( text = "    Basic reproductive rate = " + str(beta*sigma*mu*(nu+alpha*l)/(nu*(nu+sigma)*(nu+delta)*(nu+alpha))) )
scene.append_to_caption("\n\n")

graph(width=400, height=250, background = color.white, xtitle = "Time", ytitle = "Number of people", title = "SEIQRDV Model")
gcurve()
infectedDots = gdots(interval = 10, color=color.red, label = "Infected", legend = False, radius = 0.5)
exposedDots = gdots(interval = 10, color = color.blue, label = "Exposed", legend = False, radius = 0.5)
recoveredDots = gdots(interval = 10, color=color.green, label = "Recovered", legend = False, radius = 0.5)
susceptibleDots = gdots(interval = 10, color = color.yellow, label = "Susceptible", legend = False, radius = 0.5)
vaccinatedDots = gdots(interval = 10, color = color.purple, label = "Vaccinated", legend = True, radius = 0.5)
deadDots = gdots(interval = 10, color = color.black, label = "Dead", legend = False, radius = 0.5)
quarantinedDots = gdots(interval = 10, color = color.orange, label = "Quarantined", legend = True, radius = 0.5)

# Start the simulation :
button(text = "Play/Stop", bind = run)
while ( abs(- beta * S * I / N + eta * R - alpha*S - nu * S + mu * N) > tol ) :
    rate(1000)
    
    if running :
        dS = - beta * S * I / N + eta * R - alpha*S - nu * S + mu * N 
        dE = beta * S * I / N - sigma * E - nu * E + l * beta * V * I / N
        dI = sigma * E - delta * I - nu * I 
        dR = (1-k) * gamma * Q - nu * R 
        dD = k * rho * Q 
        dV = alpha * S - nu * V - l * beta * V * I / N 
        dQ = delta * I - (1-k) * gamma * Q - k * rho * Q - nu * Q 
    
        S += dS * dt
        E += dE * dt
        I += dI * dt
        R += dR * dt 
        V += dV * dt
        Q += dQ * dt
        D += dD * dt
    
        infectedDots.plot(t, I, min = 0,max = N)
        exposedDots.plot(t,E, min = 0, max = N)
        recoveredDots.plot(t, R, min = 0,max = N)
        susceptibleDots.plot(t, S, min = 0,max = N)
        vaccinatedDots.plot(t, V, min = 0, max = N)
        quarantinedDots.plot(t, Q, min = 0, max = N)
        deadDots.plot(t, D, min = 0, max = N)
        
        t += dt

print("End of SEIQRDV Model.\n")

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

## CONCLUSION 

Parler des limites de R0 : mutation
Introduire les models stochastiques

Citer mes sources.
Verifier mes simulation 
1. SIR :
2. SEIRS : verifier exactitude du modele
3. Saoudia : verif


Thanks for the course.

#### References :
- https://www.aimspress.com/article/id/3989
- https://docs.idmod.org/projects/emod-hiv/en/latest/model-seir.html#seirs-model
- https://www.nature.com/articles/s41592-020-0856-2
- On a Discrete SEIR Epidemic Model with Exposed Infectivity, Feedback Vaccination and Partial Delayed Re-Susceptibility
- https://hal.archives-ouvertes.fr/hal-01758940/document

In [25]:
1./10 

0.1